In [29]:
import requests
import pandas as pd 
from sqlalchemy import create_engine , text
from dotenv import load_dotenv
import os

In [30]:
load_dotenv()

True

In [31]:
POSTGRES_URI = os.getenv('POSTGRES_URI') or " "
engine = create_engine(POSTGRES_URI)

In [32]:
players_df = pd.read_sql('SELECT * FROM players', con=engine)

In [33]:
player_id_array = players_df['player_id'].tolist()

In [34]:
# df = pd.DataFrame()

In [35]:
# j = 0
# for i in player_id_array:
#     url = f"https://fantasy.premierleague.com/api/element-summary/{i}/"
#     response = requests.get(url)
#     data = response.json()
#     df = pd.concat([df, pd.DataFrame(data['history'])], ignore_index=True)
#     j += 1 
#     print(j)

df = pd.read_csv("fact_table_data.csv")

In [36]:
df = df[["element", "fixture", "opponent_team", "total_points", "was_home", "kickoff_time", "team_a_score", "team_h_score", "round", "goals_scored", "assists", "clean_sheets", "goals_conceded", "own_goals", "penalties_saved", "penalties_missed", "yellow_cards", "red_cards", "saves", "bonus", "bps"]]

In [37]:
df.rename(
    columns={
        "element": "player_id",
        "fixture" : "fixture_id",
        "opponent_team" : "opponent_team_id"
    },
    inplace=True
)

In [38]:
df_fixtures = pd.read_sql('SELECT * FROM fixture', con=engine)

In [39]:
fixture_array = df["fixture_id"].tolist()
event_array = []
for i in fixture_array:
    event = df_fixtures[df_fixtures['fixture_id'] == i]["event"].values[0]
    event_array.append(event)

In [44]:
df["event_id"] = event_array

In [46]:
phase_events = {
    1: list(range(1, 3 + 1)),
    2: list(range(4, 6 + 1)),  
    3: list(range(7, 9 + 1)),   
    4: list(range(10, 13 + 1)), 
    5: list(range(14, 19 + 1)),  
    6: list(range(20, 24 + 1)),  
    7: list(range(25, 28 + 1)),  
    8: list(range(29, 31 + 1)), 
    9: list(range(32, 34 + 1)),  
    10: list(range(35, 38 + 1))  
}

def assign_phase(event_id):
    for phase, events in phase_events.items():
        if event_id in events:
            return phase    
    return None

phase_array = []

for i in event_array:
    phase = assign_phase(i)
    phase_array.append(phase)
    
df["phase_id"] = phase_array

In [47]:
df.head()

,player_id,fixture_id,opponent_team_id,total_points,was_home,kickoff_time,team_a_score,team_h_score,round,goals_scored,...,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,event_id,phase_id
0,1,9,14,10,False,2025-08-17T15:30:00Z,1.0,0.0,1,0,...,0,0,0,1,0,7,3,38,11,4
1,1,11,11,6,True,2025-08-23T16:30:00Z,0.0,5.0,2,0,...,0,0,0,0,0,1,0,28,9,3
2,1,25,12,2,False,2025-08-31T15:30:00Z,0.0,1.0,3,0,...,0,0,0,0,0,2,0,12,20,6
3,1,31,16,6,True,2025-09-13T11:30:00Z,0.0,3.0,4,0,...,0,0,0,0,0,1,0,24,14,5
4,1,41,13,2,True,2025-09-21T15:30:00Z,1.0,1.0,5,0,...,0,0,0,0,0,2,0,13,11,4
